# API for Google Maps
## Case: "Private Banking Advisers at BCB Edmonton" ([UVA-QA-0905](http://store.darden.virginia.edu/private-banking-advisers-at-bcb-edmonton-a), [0906](http://store.darden.virginia.edu/private-banking-advisers-at-bcb-edmonton-b-visualizing-the-business), [0907](http://store.darden.virginia.edu/private-banking-advisers-at-bcb-edmonton-c-calculating-travel-times))

[Application Programming Interface (API)](https://en.wikipedia.org/wiki/API) allows one program communicate with another. 

Recall the BCB Edmonton case: the case protagonist needs to determine how many private bankers advisers she needs for the region, where should they be located, and which of the 35 branches should be covered by which advisor. A sub-task to this problem is to determine how long it will take an advisor with a home base at branch 'i' to travel to branch 'j' and back to cover for the appointments at 'j'. 

How would you go about finding the travel time from point A to point B in your daily life? Most likely, you would use a mapping app on your computer or smartphone -- Google Maps, Bing Maps, Yandex Maps, or the like. You would type the origin address (that of point A), the destination address (that of point B), the travel mode (walking, driving, etc.) and the time (now, or some time in the future). The app would then give you the estimated travel time. 

The code below does just that for 35x35 = 1,225 origin-destination pairs corresponding to the BCB branches in Edmonton.

<b> Note: </b> To use the code you will need to obtain the Google Maps Distance Matrix API key, which can be requested here https://developers.google.com/maps/documentation/distance-matrix/get-api-key#key You will need to enter your credit card number, but the personal use of the API within the scope of this case should not lead to any cost. Do not share your API key with others (if many people use your key, you may be billed).

In [ ]:
# Start by importing (loading) the necessary packages and libraries

import json
import requests
import csv
import pandas as pd

In [ ]:
# This block defines several custom functions which are necessary since Google Maps API only allows 25 origin-destination pairs at a time
# 1) API_response function is actually interacting with Google Maps API
# 2) chunks function is splitting the larger set of origin-destinations into chunks of less than 25
# 3) getDistance function is using the API_response function with chunks function to obtain the results for all the origin-destination pairs

def API_response(origin,destinations,time):

    base_url = 'https://maps.googleapis.com/maps/api/distancematrix/json?'

    # your own Google Maps Distance Matrix API key goes here: 
    #api_key = '[*]' 

    parameters = {'origins': origin,
            'destinations' : '|'.join(destinations),
            'mode' : 'driving',
            'key' : api_key,
            'departure_time' : time }

    r = requests.get(base_url, params = parameters)

    r = r.json()

    results = r['rows'][0]['elements']

    # Google only allows a certain number of API calls per second (to stay within the free API)
    # This artifically slows the process to allow the whole matrix to be processed
    i = 0
    while i < 10000000:
        i = i + 1
    
    return results

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

       
def getDistance(full_origins,destinations,time):

    new_dict = {}

    for origin in full_origins:

        new_list = []

        for sub_destinations in chunks(destinations, 24):
            results = API_response(origin, sub_destinations, time)
            new_list.extend([result['duration']['text'] for result in results])

        new_dict[origin] = new_list

    return new_dict

In [ ]:
# Google Maps API uses UNIX time -- the number of seconds since Jan. 1, 1970
# For example, Dec 7, 2020 at 9am EST is 1607331600

# However, according to the API documentation, datetime cannot be in the past; it must be "now" or in the future
# The code below will set the time to be one hour (3600 seconds) after the time when this code is ran
# Alternatively students could use any the desired departure time (in the form of a UNIX timestamp integer) 

from datetime import datetime

time = int(datetime.timestamp(datetime.now())) + 3600 
time


1638037730

In [ ]:
# Read the (A) case data file (converted into the CSV) and process the PostalCode column by removing the space

inputs_df = pd.read_excel("/content/qa-0905x.xlsx", header = 0)

origins = list(inputs_df["PostalCode"].str.split(" ").str.join('')) # this is to remove the space between the postal codes

# Duplicate the list of origins to creat the list of destinations 
destinations = origins.copy()

print(origins)
print(destinations)

['T4X0B6', 'T5A5C1', 'T5B0S1', 'T5E4C6', 'T5J1V7', 'T5J3N3', 'T5K0M8', 'T5L4Z6', 'T5M2L7', 'T5M3L7', 'T5T1K8', 'T5T5L5', 'T5Y0L2', 'T5Z3J8', 'T6B0P2', 'T6E2A3', 'T6E5A7', 'T6H5T1', 'T6J6P9', 'T6K4B4', 'T6R0G4', 'T6T0C2', 'T6V1J6', 'T6W1A2', 'T6X0P2', 'T7X2K6', 'T7Z2W7', 'T8A4N5', 'T8A5V9', 'T8H1S8', 'T8L3W2', 'T8N3L3', 'T8N5T8', 'T8R1R4', 'T9E6Z7']
['T4X0B6', 'T5A5C1', 'T5B0S1', 'T5E4C6', 'T5J1V7', 'T5J3N3', 'T5K0M8', 'T5L4Z6', 'T5M2L7', 'T5M3L7', 'T5T1K8', 'T5T5L5', 'T5Y0L2', 'T5Z3J8', 'T6B0P2', 'T6E2A3', 'T6E5A7', 'T6H5T1', 'T6J6P9', 'T6K4B4', 'T6R0G4', 'T6T0C2', 'T6V1J6', 'T6W1A2', 'T6X0P2', 'T7X2K6', 'T7Z2W7', 'T8A4N5', 'T8A5V9', 'T8H1S8', 'T8L3W2', 'T8N3L3', 'T8N5T8', 'T8R1R4', 'T9E6Z7']


In [ ]:
# Visualize the result of the 'chunks' function: the list of 35 is split into two: one with 24 elements and another with 11

print(list(chunks(destinations, 24)))

[['T4X0B6', 'T5A5C1', 'T5B0S1', 'T5E4C6', 'T5J1V7', 'T5J3N3', 'T5K0M8', 'T5L4Z6', 'T5M2L7', 'T5M3L7', 'T5T1K8', 'T5T5L5', 'T5Y0L2', 'T5Z3J8', 'T6B0P2', 'T6E2A3', 'T6E5A7', 'T6H5T1', 'T6J6P9', 'T6K4B4', 'T6R0G4', 'T6T0C2', 'T6V1J6', 'T6W1A2'], ['T6X0P2', 'T7X2K6', 'T7Z2W7', 'T8A4N5', 'T8A5V9', 'T8H1S8', 'T8L3W2', 'T8N3L3', 'T8N5T8', 'T8R1R4', 'T9E6Z7']]


In [ ]:
# Call the function getDistance, which will obtain the results, and print them for visual inspection

result = getDistance(origins, destinations, time)

print(result)

In [ ]:
# Convert the result to a pandas dataframe for easier manipulation

result_pd = pd.DataFrame(result)   

# Remove "mins" from each entry result and convert the numeric values from strings to numbers

for col in result_pd:
    result_pd[col] = result_pd[col].str.split(' ').str[0].astype('float') 

result_pd.index = result_pd.columns

In [ ]:
# Finally, write the resultant matrix of travel times to a csv file

result_pd.to_csv('Matrix_of_travel_times.csv')